In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import xarray as xr

from canopy_app import config_cases, run, run_config_sens, DEFAULT_POINT_INPUT

xr.set_options(display_expand_data=False, display_expand_attrs=False)

plt.rcParams.update(
    {
        "figure.autolayout": True,
        "axes.formatter.limits": (-3, 4),  # scilimits
        "axes.formatter.use_mathtext": True,
        "figure.max_open_warning": 0,
    }
)

%matplotlib widget

## Default case (2-D)

Southeast US

In [ ]:
inp = xr.open_dataset("../input/gfs.t12z.20220701.sfcf000.canopy.nc")

In [ ]:
inp

In [ ]:
%%time

ds = run()

In [ ]:
ds.waf.plot(x="lon", y="lat", size=3)

In [ ]:
proj = ccrs.Mercator()
tran = ccrs.PlateCarree()

fig = plt.figure(constrained_layout=True)
ax = fig.add_subplot(projection=proj)

ax.coastlines()
ax.gridlines(draw_labels=True)

ds.waf.plot(x="lon", y="lat", ax=ax, transform=tran, alpha=0.8)

## Default point case

In [ ]:
DEFAULT_POINT_INPUT.T

In [ ]:
ds = run(
    config={
        "filenames": {"file_vars": "../input/input_variables_point.txt"},
        "userdefs": {"infmt_opt": 1, "nlat": 1, "nlon": 1},
    },
)

In [ ]:
ds.canopy_height

In [ ]:
(
    ds[[vn for vn in ds.data_vars if vn.startswith("emi_")]]
    .sel(z=10, method="nearest")
    .to_pandas()
    .to_frame()
    .plot.bar(figsize=(6, 3), legend=False)
)